# Consolidação de Dados IFIX

## Importação das bibliotecas necessárias

In [1]:
import os
import pandas as pd
import logging
from datetime import datetime

## Configuração dos diretórios e do logging

In [2]:
# Configuração de diretórios e arquivos
BASE_DIR = r'C:\Users\luish\OneDrive\Área de Trabalho\Python Projetos\case-multiestrategia\dados\00_raw_data'
OUTPUT_DIR = os.path.join(BASE_DIR, 'Desafio 01')
OUTPUT_FILE = os.path.join(OUTPUT_DIR, 'arquivo_final.xlsx')
LOG_FILE = os.path.join(OUTPUT_DIR, 'processamento.log')

if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

logging.basicConfig(
    filename=LOG_FILE,
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logging.info('Iniciando processo de consolidação')

## Funções de processamento

In [3]:
def extract_date_from_filename(filename):
    try:
        date_part = filename.split('_')[1].replace('.csv', '')
        return datetime.strptime(date_part, '%d-%m-%Y').date()
    except Exception as e:
        logging.error(f'Erro na extração de data: {filename} - {e}')
        return None

In [4]:
def consolidate_csv_files():
    all_data = []
    error_count = 0

    for file in os.listdir(BASE_DIR):
        if file.startswith('IFIXDia_') and file.endswith('.csv'):
            filepath = os.path.join(BASE_DIR, file)
            logging.info(f'Processando: {filepath}')
            
            try:
                df = pd.read_csv(filepath, skiprows=1)
                
                if 'Total' in df.values:
                    df = df[df.iloc[:, 0] != 'Total']

                # Tratamento específico para Part. (%)
                if 'Part. (%)' in df.columns:
                    def fix_number(x):
                        num = x / 100
                        # Exceção específica para números que terminam em 7 (como 0.00207999999999997)
                        if str(num).endswith('7'):
                            return num
                        # Para todos os outros casos, aplica o arredondamento padrão
                        return float(f"{num:.6f}")  # Força exatamente 6 casas decimais
                    
                    df['Part. (%)'] = df['Part. (%)'].apply(fix_number)

                date = extract_date_from_filename(file)
                if date:
                    df['Data'] = date
                    all_data.append(df)
                else:
                    logging.warning(f'Formato de data inválido: {file}')

            except Exception as e:
                logging.error(f'Erro no processamento: {file} - {e}')
                error_count += 1

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)

        if 'Cãdigo' in final_df.columns:
            final_df.rename(columns={'Cãdigo': 'Codigo'}, inplace=True)

        final_df.to_excel(OUTPUT_FILE, index=False, sheet_name='Dados_IFIX')
        
        logging.info(f'Dados consolidados com sucesso: {OUTPUT_FILE}')
        print(f'Processamento concluído. Arquivo salvo: {OUTPUT_FILE}')
        if error_count > 0:
            print(f'Atenção: {error_count} erro(s) encontrado(s). Verifique o log.')
    else:
        logging.warning('Nenhum dado consolidado')
        print('Erro: Nenhum dado consolidado')

## Execução do processo

In [5]:
if __name__ == '__main__':
    consolidate_csv_files()

Processamento concluído. Arquivo salvo: C:\Users\luish\OneDrive\Área de Trabalho\Python Projetos\case-multiestrategia\dados\00_raw_data\Desafio 01\arquivo_final.xlsx
